# Behavioral Preprocessing

Functions included: Distance between defeated and bullying mouse; Defeated mouse head direction; Behavioral annotation.

In [2]:
import numpy as np 
import os
import glob
import pandas as pd
import h5py
from math import degrees, atan2

In [7]:
os.chdir("/home/donghan/DeepLabCut/data/rotated") #Change to your own directory

## Data read from h5 file

In [18]:
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(s[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(s[start:end] + '.csv')

## Distance bewteen defeated mouse head and encloser
### The Pythagorean theorem

In [31]:
#Distance bewteen defeated mouse head and encloser
#Encloser coordination axis need modification #IMPORTANT

def cal_dist(data): 
    fixedX = 50
    fixedY = 200
    x = data["x"]
    y = data["y"]
    dist = np.sqrt((x - fixedX)**2 + (y - fixedY)**2)
    if __name__ == "__main__": 
        return dist  

## Head Direction

In [34]:
#Head Direction

def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p2.x - p1.x
    yDiff = p2.y - p1.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.

## Annotations

Based on pre-setting video, the coordination axis could be used in annotating. 

For example, since the behavioral camera is not perfectly match the size of the cage, we have the interaction zone of four points, (70, 116), (70, 307), (171, 116), (171, 307), so once the mouse's head enter this area, the behavior will be labled as "1"

In [38]:
#Annotations of bahavior definition

# Approaching the interaction zone (area around the enclosure)
# Avoiding the interaction zone - this could include instances where the animal enters the 1/2 of the chamber containing the enclosure + the head direction is facing the enclosure, but it doesn't enter the interaction zone
# Sniffing/directly interacting with the enclosure
# Sitting in the corners (2 corners closest to enclosure vs 2 far corners)

def annotate(interactionXL = 70, interactionXR = 171, interactionYUpper = 307, interactionYLower = 116, enclosureX = 134, enclosureYLower = 146, enclosureYUpper = 256, corX = 315, corUppRY = 302, corLowRY = 124, MidLineX = 190, data = data):
    posX = data['x'].tolist()
    posY = data['y'].tolist()
    head = head_dir(data)[0] 
    #Whether the head is facing enclosure
    label = []
    for i in range(0, len(posX)):
        if posX[i] < interactionXR and posY[i] < interactionYUpper and posY[i] > interactionYLower: 
            #Approaching
            label.append(1)
        elif posX[i] <= MidLineX and posX[i] > interactionXR and (posY[i] > interactionYUpper or posY[i] < interactionYLower): 
            #Avoidance
            label.append(2)
        elif posX[i] < enclosureX and posY[i] > enclosureYLower and posY[i] < enclosureYUpper: 
            #Sniffing
            label.append(3)
        elif (posX[i] > corX and posY[i] < corLowRY) or (posX[i] > corX and posY[i] > corUppRY): 
            #Sitting in the corners
            label.append(4)
        else: #Other cases
            label.append(5)
    if __name__ == "__main__": 
        return label

#### Remove the first ten seconds, which has no mouse in the case (~200 frames)

In [25]:
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[200:] 
    #There are no mouse in the first ten seconds
    cal_dist(move_data) 
    #Distance
    head_dir(move_data) 
    #Head direction
    annotate(data = move_data) 
    #Annotation

## Appendix
### Video Rotation

In [9]:
import numpy as np
import cv2
import glob
os.chdir("/home/donghan/DeepLabCut/data") 
#Working directory that stores video data
def rotate(image, angle, center=None, scale=1): 
    #scale = 1: original size
    rows,cols,ch = image.shape
    if center == None:
        center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale) 
    #Matrix: Rotate with center by angles
    dst = cv2.warpAffine(image,M,(cols,rows)) 
    #After rotation
    return dst


def videorotate(filename, output_name, display_video = False):
    # capture video
    cap = cv2.VideoCapture(filename)

    #read video frame by frame
    #extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))

    #Make a directory to store the rotated videos
    path = "./rotated"
    try:  
        os.mkdir(path)
    except OSError:  
        pass
    else:  
        print ("Successfully created the directory %s " % path)
        
    #Automatically name the rotated videos  
    file = "./rotated/" + output_name
    out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz) 
    #Integrate all frames to video

    
    #Read videos and rotate by certain degrees
    while(cap.isOpened()):
        #flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = rotate(img, -4.5) 
            #Flipped Vertically
            out.write(img2)
            if display_video == True:
                cv2.imshow('rotated video',img2) 

            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            print (filename, 'successfully rotated!!!' )
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    
# #Generate all rotating videos    
# filenames = glob.glob('*.mp4') #Return the file name with .mp4 extention 
# for i in filenames:
#     videorotate(i,os.path.splitext(i)[0] + " rotated.mp4")
    
videorotate("behavCam1.avi", "behavCam1 rotated.avi")

behavCam1.avi successfully rotated!!!
